In [ ]:
##############################

# 1. Single Tracking 

##############################

import cv2
import sys
import os
import numpy as np

i = int(input("Please choose a number between 0 and 6 !\nNumber : "))

if i > 6:
    print("Not Existing Tracking Method!")
    
tracker_types = ["BOOSTING","MIL","KCF","TLD","MEDIANFLOW","MOSSE","CSRT"]
tracker_type = tracker_types[i]

print("Tracker Type : ", tracker_type)

if tracker_type == "BOOSTING":
    tracker = cv2.legacy.TrackerBoosting_create()

elif tracker_type == "MIL":
    tracker = cv2.legacy.TrackerMIL_create()

elif tracker_type == "KCF":
    tracker = cv2.legacy.TrackerKCF_create()
    
elif tracker_type == "TLD":
    tracker = cv2.legacy.TrackerTLD_create()

elif tracker_type == "MEDIANFLOW":
    tracker = cv2.legacy.TrackerMedianFlow_create()

elif tracker_type == "MOSSE":
    tracker = cv2.legacy.TrackerMOSSE_create()
    
elif tracker_type == "CSRT":
    tracker = cv2.legacy.TrackerCSRT_create()
    
#print(tracker)

cap = cv2.VideoCapture("Videos/race.mp4")

if not cap.isOpened():
    print("Error While Loading The Video ! ")
    sys.exit()

conn, frame = cap.read()

if not conn:
    print("Error While Loading The Frame !")
    sys.exit()
    
#print("Connect : ",conn)

bbox = cv2.selectROI(frame) # Region Of Interest
#print("Box : ", bbox)

tracker.init(frame,bbox)

colors = (np.random.randint(0,255), np.random.randint(0,255), np.random.randint(0,255))
print("Color : " ,colors)

while True:
    
    conn,frame = cap.read()
    
    if not conn:
        
        break
        
    ok, bbox = tracker.update(frame)
    
    if ok == True:
        
        (x,y,w,h) = [int(v) for v in bbox]
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),colors,2)
        
    else:
        
        cv2.putText(frame, "Tracking Failure!", (500,40), cv2.FONT_HERSHEY_COMPLEX, .75, (0,0,255))
        
    
    cv2.putText(frame, f"Tracker : {tracker_type}",(500,80),cv2.FONT_HERSHEY_COMPLEX, .75, (0,0,255))
    

    cv2.imshow("Tracking", frame)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

In [1]:
##############################

# 2. Multiple Tracking 

##############################

import cv2
from numpy.random import randint

tracker_types = ["BOOSTING","MIL","KCF","TLD","MEDIANFLOW","MOSSE","CSRT"]

def create_tracker_by_name(tracker_type):
    
    if tracker_type == tracker_types[0]:
        tracker = cv2.legacy.TrackerBoosting_create()
        
    elif tracker_type == tracker_types[1]:
        tracker = cv2.legacy.TrackerMIL_create()
    
    elif tracker_type == tracker_types[2]:
        tracker = cv2.legacy.TrackerKCF_create()
    
    elif tracker_type == tracker_types[3]:
        tracker = cv2.legacy.TrackerTLD_create()
    
    elif tracker_type == tracker_types[4]:
        tracker = cv2.legacy.TrackerMedianFlow_create()

    elif tracker_type == tracker_types[5]:
        tracker = cv2.legacy.TrackerMOSSE_create()
        
    elif tracker_type == tracker_types[6]:
        tracker = cv2.legacy.TrackerCSRT_create()
    
    else:
        
        tracker = None
        print("Invalid Name! Avaiable Trackers: \n")
        for t in tracker_types:
            print(t)
        print("""""")
        
    return tracker

#print(create_tracker_by_name("CSRT"))
#print(create_tracker_by_name("CSRT8"))

cap = cv2.VideoCapture("Videos/race.mp4")

if not cap.isOpened():
    print("Error While Loading The Video!")
    sys.exit()
    
conn, frame = cap.read()
bboxes = []
colors = []

while True:
    
    bbox = cv2.selectROI("MultiTracker",frame)
    bboxes.append(bbox)
    colors.append( ( randint(0,255), randint(0,255), randint(0,255) ) )
    
    print("Press Q to quit and start Tracking")
    print("Press Any Other Key To Select The Next Object")
    
    k = cv2.waitKey(0) & 0xFF
    if k == ord("q"): # Q-quit
        break

print()
print("Boxes : ", bboxes)
print("Colors : ", colors)

tracker_type = "CSRT"
multi_tracker = cv2.legacy.MultiTracker_create()

for bbox in bboxes:
    multi_tracker.add(create_tracker_by_name(tracker_type), frame, bbox)
    
while cap.isOpened():
    
    conn, frame = cap.read()
    
    if not conn:
        break
        
    ok, boxes = multi_tracker.update(frame)  
    for i,new_box in enumerate(boxes):
        
        (x,y,w,h) = [int(v) for v in new_box]
        cv2.rectangle(frame, (x,y), (x+w, y+h), colors[i], 2)
        
    cv2.imshow("MultiTracker",frame)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break
        
        
cap.release()
cv2.destroyAllWindows()

Press Q to quit and start Tracking
Press Any Other Key To Select The Next Object
Press Q to quit and start Tracking
Press Any Other Key To Select The Next Object
Press Q to quit and start Tracking
Press Any Other Key To Select The Next Object
Press Q to quit and start Tracking
Press Any Other Key To Select The Next Object

Boxes :  [(383, 250, 140, 212), (644, 239, 103, 216), (826, 224, 105, 202), (948, 236, 110, 213)]
Colors :  [(241, 137, 39), (187, 177, 110), (15, 232, 56), (74, 135, 217)]


In [7]:
##############################

# 3. Goturn Tracking 

##############################


import cv2
import sys
import os
from numpy.random import randint

if not (os.path.isfile("goturn.caffemodel") and os.path.isfile("goturn.prototxt")):
    print("Error Loading The Network Files !")
    sys.exit()
    
tracker = cv2.TrackerGOTURN_create()
tracker_type = type(tracker).__name__

cap = cv2.VideoCapture("Videos/race.mp4")

if not cap.isOpened():
    print("Error While Loading The Video !")
    sys.exit()
    
conn,frame = cap.read()

if not conn:
    print("Error While Loading The Frame !")
    sys.exit()
    
bbox = cv2.selectROI(frame)

tracker.init(frame,bbox)

colors = ( randint(0,255),randint(0,255),randint(0,255))

while True:
    
    conn,frame = cap.read()
    
    if not conn:
        break
        
        
    ok,bbox = tracker.update(frame)
    
    if ok == True:
        
        (x,y,w,h) = [int(v) for v in bbox]
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),colors,2)
        
    else:
        
        cv2.putText(frame, "Tracking Failure!", (500,40), cv2.FONT_HERSHEY_COMPLEX, .75, (0,0,255))
        
    
    cv2.putText(frame, f"Tracker : {tracker_type}",(500,80),cv2.FONT_HERSHEY_COMPLEX, .75, (0,0,255))
    

    cv2.imshow("Tracking", frame)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break
        
cap.release()
cv2.destroyAllWindows()